# WM and GM distribution in the pediatric spinal cord

This jupyter notebook includes scripts to generate figures related to the white matter and gray matter distribution in the pediatric spinal cord.

In [27]:
import os
import pandas as pd
import json
import yaml
import re
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import webbrowser
import statsmodels.formula.api as smf

### Load config file to get path to dataset 

In [28]:
# Load config file
with open('../../config/config_preprocessing.yaml' , 'r') as file:
    config = yaml.safe_load(file)

# Get data path from config file
path_data = config['path_data']

### Get the `participants.tsv` file from the dataset

In [29]:
# Get path to participants.tsv file
participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')
participants_tsv

,participant_id,age,sex,group,scan_series,height,weight
0,sub-101,17,M,control,complete,1.778004,68.038864
1,sub-102,15,F,control,complete,1.625603,52.163129
2,sub-103,15,M,control,complete,1.651003,54.431091
3,sub-104,15,F,control,complete,1.625603,52.163129
4,sub-105,13,M,control,complete,1.524000,35.381000
...,...,...,...,...,...,...,...
110,sub-214,6,M,control,complete,NaN,NaN
111,sub-215,16,F,control,complete,NaN,NaN
112,sub-216,15,F,control,complete,NaN,NaN
113,sub-217,15,M,control,complete,NaN,NaN


## Dataframe of subjects included in the pipeline analysis

The following dataframe contains only the subjects that were included in this pipeline analysis.

In [30]:
def get_list_of_subjects_to_include(contrast, path_data, missing_data_subjects):
    """
    This function takes an image contrast (T2w, dwi, etc.), a path to a dataset, and a list of subjects with missing data,
    and returns a list of subjects to include in the analysis.

    The dataset needs to be in BIDS format, and the function will look for the participants.tsv file to get the list of subjects.
    The dataset should also contain an `exclude.yml` file that lists subjects to exclude from the analysis.
    """

    # Get the `participants.tsv` file and read it into a dataframe
    participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')

    # Get all subject IDs from the participants.tsv
    all_subjects = participants_tsv['participant_id'].tolist()

    # Get list of subjects to exclude from the analysis from the `exclude.yml` file
    with open(os.path.join(path_data, 'exclude.yml'), 'r') as file:
        exclude_yml = yaml.safe_load(file)

    exclude_t2star_key = exclude_yml.get(contrast, []) # Extract subjects under contrast key
    exclude_subjects = sorted(set(re.match(r"(sub-\d+)", entry).group(1) for entry in exclude_t2star_key if re.match(r"(sub-\d+)", entry))) # Extract the subject ID 

    # Add the list of subjects with missing data to the exclude_subjects list
    exclude_subjects.extend(missing_data_subjects)
    
    # Remove duplicates (if any), sort and print the list of subjects to exclude from the analysis
    exclude_subjects = sorted(set(exclude_subjects))
    print(f'subjects to exclude : {exclude_subjects}')

    # Compute the list of subjects to include in the analysis 
    include_subjects = [sub for sub in all_subjects if sub not in exclude_subjects]

    # Convert the list of included subjects to a dataframe
    include_subjects = participants_tsv[participants_tsv['participant_id'].isin(include_subjects)]

    return include_subjects

In [31]:
# List of subjects with missing t2star data or missing T2w rootlet segmentation
missing_t2star_subjects = [
                        "sub-109",
                        "sub-125",
                        "sub-136",
                        "sub-152",
                        "sub-159",
                        "sub-174",
                        "sub-198",
                        "sub-200",
                        "sub-205",
                        "sub-211",
                        "sub-213"]

missing_rootlets_subjects = ["sub-108", "sub-110", "sub-111", "sub-121", "sub-133", "sub-136", "sub-141", "sub-150", 
                             "sub-159", "sub-160", "sub-161", "sub-163", "sub-165", "sub-167", "sub-168", "sub-169", 
                             "sub-170", "sub-177", "sub-179", "sub-186", "sub-190", "sub-191", "sub-193", "sub-198", 
                             "sub-203", "sub-208", "sub-209", "sub-211", "sub-213", "sub-214"]

# Combine both lists of missing subjects
missing_subjects = missing_t2star_subjects + missing_rootlets_subjects
print(missing_subjects)

# Get the list of subjects to include in the analysis
include_t2star_subjects = get_list_of_subjects_to_include('t2starw', path_data, missing_subjects)
include_t2star_subjects.to_csv(os.path.join('../tables/WMGM_distribution/include_t2star_subjects.csv'), sep='\t', index=False)

print(include_t2star_subjects.shape[0], "subjects to include in the analysis")
print(f"\n list of subjects to include : \n {include_t2star_subjects['participant_id'].tolist()}")

include_t2star_subjects

['sub-109', 'sub-125', 'sub-136', 'sub-152', 'sub-159', 'sub-174', 'sub-198', 'sub-200', 'sub-205', 'sub-211', 'sub-213', 'sub-108', 'sub-110', 'sub-111', 'sub-121', 'sub-133', 'sub-136', 'sub-141', 'sub-150', 'sub-159', 'sub-160', 'sub-161', 'sub-163', 'sub-165', 'sub-167', 'sub-168', 'sub-169', 'sub-170', 'sub-177', 'sub-179', 'sub-186', 'sub-190', 'sub-191', 'sub-193', 'sub-198', 'sub-203', 'sub-208', 'sub-209', 'sub-211', 'sub-213', 'sub-214']
subjects to exclude : ['sub-106', 'sub-107', 'sub-108', 'sub-109', 'sub-110', 'sub-111', 'sub-120', 'sub-121', 'sub-124', 'sub-125', 'sub-133', 'sub-136', 'sub-139', 'sub-141', 'sub-144', 'sub-150', 'sub-152', 'sub-154', 'sub-159', 'sub-160', 'sub-161', 'sub-163', 'sub-165', 'sub-167', 'sub-168', 'sub-169', 'sub-170', 'sub-171', 'sub-172', 'sub-174', 'sub-177', 'sub-179', 'sub-181', 'sub-186', 'sub-188', 'sub-189', 'sub-190', 'sub-191', 'sub-193', 'sub-194', 'sub-196', 'sub-198', 'sub-199', 'sub-200', 'sub-203', 'sub-204', 'sub-205', 'sub-208

,participant_id,age,sex,group,scan_series,height,weight
0,sub-101,17,M,control,complete,1.778004,68.038864
1,sub-102,15,F,control,complete,1.625603,52.163129
2,sub-103,15,M,control,complete,1.651003,54.431091
3,sub-104,15,F,control,complete,1.625603,52.163129
4,sub-105,13,M,control,complete,1.524000,35.381000
...,...,...,...,...,...,...,...
106,sub-210,6,F,control,complete,NaN,NaN
111,sub-215,16,F,control,complete,NaN,NaN
112,sub-216,15,F,control,complete,NaN,NaN
113,sub-217,15,M,control,complete,NaN,NaN


## Plot demographics

This function plots the age and sex distribution of the subjects included in a pipeline analysis, according to the include list generated above. 

In [32]:
def plot_demographics(df):
    """
    This function plots the demographic information of participants, given a dataframe with the list of subjects to include in the analysis.
    """

    # Sort by sex
    df_M = df[df['sex'] == 'M']
    df_F = df[df['sex'] == 'F']

    # Round down age to nearest month 
    df['age'] = np.floor(df['age']) 

    # Create subplot
    fig = make_subplots(rows=1, cols=1)

    # Add histogram for female subjects
    fig.add_trace(go.Histogram(
        x=df_F['age'], 
        name='F', 
        marker=dict(color= "#D19D88"),
        opacity=1.0,
        legendgroup='F',
        ),
        row=1, col=1
    )

    # Add histogram for male subjects
    fig.add_trace(go.Histogram(
        x=df_M['age'], 
        name='M', 
        
        marker=dict(color="#5C8EA1"),
        opacity=1.0,
        legendgroup='M',
        ), 
        row=1, col=1
    )

    # Define age tick range
    tick_vals = list(range(6, 18)) 

    # Update layout
    fig.update_layout(
        width=900,
        height=500,
        font=dict(family='Arial', size=20, color='black'), 
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.0, 
            xanchor="center",  
            x=0.5,
        ),
        xaxis=dict(
            range=[5, 18],  # Set x-axis range from 6 to 17
        ),
        plot_bgcolor='white',
        barmode='stack',
        bargap=0.3,  
        xaxis_title='Age (years)',
        xaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        yaxis_title='Number of Subjects',
        yaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        xaxis_title_standoff=50, 
    )

    fig.update_xaxes(
        tickmode='array',
        tickvals=tick_vals,
        showgrid=False,
        gridwidth=1
    )

    fig.update_yaxes(
        showgrid=True,             # Horizontal grid lines
        gridcolor='lightgrey',
        gridwidth=1
    )

    # Set bin size to 1 year
    fig.update_traces(xbins=dict(size=1))

    fig.show()

In [33]:
# Plot demographics for included subjects in DWI analysis
plot_demographics(include_t2star_subjects)

# Create a dataframe for GM, WM and SC CSA (all subjects combined)

In [34]:
# Path to WM, GM and SC CSA (one file per subject)
CSA_base_folder = "../tables/WMGM_distribution/"
labels = ['WM', 'GM', 'SC']

WMGM_distribution_df = {}

for label in labels:
    label_folder = os.path.join(CSA_base_folder, label)
    label_dfs = []

    if not os.path.exists(label_folder):
        print(f"Folder not found: {label_folder}")
        continue
    
    for filename in os.listdir(label_folder):
        if filename.endswith(".csv"):
            if not any(sub in filename for sub in include_t2star_subjects['participant_id'].tolist()):
                continue  # Skip subjects not in the include list
            subject_path = os.path.join(label_folder, filename)
            df = pd.read_csv(subject_path)
            subject_id = filename.split("_")[0]  # Or use regex for more robust parsing
            df["participant_id"] = subject_id
            label_dfs.append(df)
    
    if label_dfs:
        WMGM_distribution_df[label] = pd.concat(label_dfs, ignore_index=True)
    else:
        print(f"Warning: No CSV files found for label {label}")
        WMGM_distribution_df[label] = pd.DataFrame()

# Add age and sex to DTI metric dataframe
for label in WMGM_distribution_df:
    WMGM_distribution_df[label] = WMGM_distribution_df[label].merge(include_t2star_subjects, on="participant_id", how="left")
                                                                    

In [35]:
# Save combined WM, GM and SC CSA dataframes to csv files

df_WM = WMGM_distribution_df['WM']
#WMGM_distribution_df['WM'].to_csv('../tables/WMGM_distribution/WM/WM_CSA.csv', index=False)

df_GM = WMGM_distribution_df['GM']
#WMGM_distribution_df['GM'].to_csv('../tables/WMGM_distribution/GM/GM_CSA.csv', index=False)

df_SC = WMGM_distribution_df['SC']
#WMGM_distribution_df['SC'].to_csv('../tables/WMGM_distribution/SC/SC_CSA.csv', index=False)

In [36]:
df_SC

,Timestamp,SCT Version,Filename,Slice (I->S),VertLevel,DistancePMJ,MEAN(area),STD(area),MEAN(angle_AP),STD(angle_AP),...,MEAN(solidity),STD(solidity),SUM(length),participant_id,age,sex,group,scan_series,height,weight
0,2025-10-20 09:29:48,7.1,/Users/samuellestonge/Documents/datasets/phila...,158:172,7,NaN,68.949359,1.449412,0.0,0.0,...,0.970778,0.013064,12.0,sub-101,17,M,control,complete,1.778004,68.038864
1,2025-10-20 09:29:48,7.1,/Users/samuellestonge/Documents/datasets/phila...,176:189,6,NaN,75.428600,0.932392,0.0,0.0,...,0.971104,0.012258,11.2,sub-101,17,M,control,complete,1.778004,68.038864
2,2025-10-20 09:29:48,7.1,/Users/samuellestonge/Documents/datasets/phila...,195:206,5,NaN,75.413362,1.192570,0.0,0.0,...,0.970854,0.010107,9.6,sub-101,17,M,control,complete,1.778004,68.038864
3,2025-10-20 09:29:48,7.1,/Users/samuellestonge/Documents/datasets/phila...,214:222,4,NaN,72.248916,1.260093,0.0,0.0,...,0.977410,0.004330,7.2,sub-101,17,M,control,complete,1.778004,68.038864
4,2025-10-20 09:29:48,7.1,/Users/samuellestonge/Documents/datasets/phila...,231:244,3,NaN,64.160024,21.283617,0.0,0.0,...,0.903921,0.159899,6.4,sub-101,17,M,control,complete,1.778004,68.038864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2025-10-20 09:37:13,7.1,/Users/samuellestonge/Documents/datasets/phila...,108:117,6,NaN,77.439903,1.810191,0.0,0.0,...,0.971419,0.014682,8.0,sub-192,17,F,control,complete,NaN,NaN
347,2025-10-20 09:37:13,7.1,/Users/samuellestonge/Documents/datasets/phila...,126:136,5,NaN,77.439903,1.659965,0.0,0.0,...,0.966794,0.006069,8.8,sub-192,17,F,control,complete,NaN,NaN
348,2025-10-20 09:37:13,7.1,/Users/samuellestonge/Documents/datasets/phila...,146:154,4,NaN,68.124359,0.959340,0.0,0.0,...,0.974754,0.006269,7.2,sub-192,17,F,control,complete,NaN,NaN
349,2025-10-20 09:37:13,7.1,/Users/samuellestonge/Documents/datasets/phila...,164:181,3,NaN,69.831024,1.757752,0.0,0.0,...,0.965006,0.008808,14.4,sub-192,17,F,control,complete,NaN,NaN


# Merge WM and GM dataframes and compute WM/GM ratio, GM/SC ratio and WM/SC ratio

In [37]:
# Merge WM and GM dataframes
df_GM_WM = df_WM.merge(
    df_GM,
    on=['participant_id', 'VertLevel', 'age', 'sex'], 
    suffixes=('_WM', '_GM')
)

# Add "_SC" suffix to df_SC columns except for the merge keys
df_SC = df_SC.rename(columns={col: f"{col}_SC" for col in df_SC.columns if col not in ['participant_id', 'VertLevel', 'age', 'sex']})

# Merge SC dataframe to the dataframe with WM and GM
df_GM_WM_SC = df_GM_WM.merge(
    df_SC, 
    on=['participant_id', 'VertLevel', 'age', 'sex']
)

df_GM_WM_SC.columns

Index(['Timestamp_WM', 'SCT Version_WM', 'Filename_WM', 'Slice (I->S)_WM',
       'VertLevel', 'DistancePMJ_WM', 'MEAN(area)_WM', 'STD(area)_WM',
       'MEAN(angle_AP)_WM', 'STD(angle_AP)_WM', 'MEAN(angle_RL)_WM',
       'STD(angle_RL)_WM', 'MEAN(diameter_AP)_WM', 'STD(diameter_AP)_WM',
       'MEAN(diameter_RL)_WM', 'STD(diameter_RL)_WM', 'MEAN(eccentricity)_WM',
       'STD(eccentricity)_WM', 'MEAN(orientation)_WM', 'STD(orientation)_WM',
       'MEAN(solidity)_WM', 'STD(solidity)_WM', 'SUM(length)_WM',
       'participant_id', 'age', 'sex', 'group_WM', 'scan_series_WM',
       'height_WM', 'weight_WM', 'Timestamp_GM', 'SCT Version_GM',
       'Filename_GM', 'Slice (I->S)_GM', 'DistancePMJ_GM', 'MEAN(area)_GM',
       'STD(area)_GM', 'MEAN(angle_AP)_GM', 'STD(angle_AP)_GM',
       'MEAN(angle_RL)_GM', 'STD(angle_RL)_GM', 'MEAN(diameter_AP)_GM',
       'STD(diameter_AP)_GM', 'MEAN(diameter_RL)_GM', 'STD(diameter_RL)_GM',
       'MEAN(eccentricity)_GM', 'STD(eccentricity)_GM', 'MEAN(o

In [38]:
df_GM_WM_SC

,Timestamp_WM,SCT Version_WM,Filename_WM,Slice (I->S)_WM,VertLevel,DistancePMJ_WM,MEAN(area)_WM,STD(area)_WM,MEAN(angle_AP)_WM,STD(angle_AP)_WM,...,STD(eccentricity)_SC,MEAN(orientation)_SC,STD(orientation)_SC,MEAN(solidity)_SC,STD(solidity)_SC,SUM(length)_SC,group_SC,scan_series_SC,height_SC,weight_SC
0,2025-10-20 09:33:33,7.1,/Users/samuellestonge/Documents/datasets/phila...,114:124,7,NaN,58.559890,17.383630,0.0,0.0,...,0.012760,2.267316,0.987526,0.963713,0.007920,8.800000,control,complete,NaN,NaN
1,2025-10-20 09:33:33,7.1,/Users/samuellestonge/Documents/datasets/phila...,130:138,6,NaN,56.391005,1.744757,0.0,0.0,...,0.011910,2.170168,1.150126,0.966118,0.007465,7.200000,control,complete,NaN,NaN
2,2025-10-20 09:33:33,7.1,/Users/samuellestonge/Documents/datasets/phila...,147:151,5,NaN,54.911897,0.940602,0.0,0.0,...,0.008414,1.661448,0.390163,0.955304,0.017929,4.000000,control,complete,NaN,NaN
3,2025-10-20 09:33:33,7.1,/Users/samuellestonge/Documents/datasets/phila...,164:168,4,NaN,53.887899,1.639197,0.0,0.0,...,0.015752,2.713642,0.502634,0.966815,0.009281,4.000000,control,complete,NaN,NaN
4,2025-10-20 09:33:33,7.1,/Users/samuellestonge/Documents/datasets/phila...,175:186,3,NaN,52.533235,1.292162,0.0,0.0,...,0.009076,3.863420,1.247348,0.969131,0.013534,9.600000,control,complete,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2025-10-20 09:34:50,7.1,/Users/samuellestonge/Documents/datasets/phila...,136:145,6,NaN,106.111894,28.716722,0.0,0.0,...,0.008857,3.848490,1.190725,0.964404,0.019813,8.000001,control,complete,NaN,NaN
347,2025-10-20 09:34:50,7.1,/Users/samuellestonge/Documents/datasets/phila...,154:161,5,NaN,42.159958,1.128538,0.0,0.0,...,0.011167,3.798181,1.709875,0.961941,0.013246,6.400001,control,complete,NaN,NaN
348,2025-10-20 09:34:50,7.1,/Users/samuellestonge/Documents/datasets/phila...,169:175,4,NaN,89.462768,36.557218,0.0,0.0,...,0.013047,1.180328,0.768345,0.962481,0.010477,5.600001,control,complete,NaN,NaN
349,2025-10-20 09:34:50,7.1,/Users/samuellestonge/Documents/datasets/phila...,180:195,3,NaN,47.359953,1.678092,0.0,0.0,...,0.007359,0.630312,0.484942,0.975116,0.008439,12.800002,control,complete,NaN,NaN


In [39]:
# Compute WM/GM ratio
df_GM_WM_SC['GM_WM_ratio'] = df_GM_WM_SC['MEAN(area)_GM'] / df_GM_WM_SC['MEAN(area)_WM'].values

# Compute GM/SC ratio
df_GM_WM_SC['GM_SC_ratio'] = df_GM_WM_SC['MEAN(area)_GM'] / df_GM_WM_SC['MEAN(area)_SC'].values

# Compute WM/SC ratio
df_GM_WM_SC['WM_SC_ratio'] = df_GM_WM_SC['MEAN(area)_WM'] / df_GM_WM_SC['MEAN(area)_SC'].values

## Plot GM, WM and SC CSA with age using spinal levels

In [57]:
def plot_CSA_per_label(df, label_name):
    """
    This function plots the CSA for a single label (e.g., "WM") across different spinal (or vertebral) levels as a function of age.
    It also fits an OLS regression and displays confidence intervals and p-values.

    Args:
        df (pd.DataFrame): DataFrame containing the data to plot. 
                           Must include columns 'age', 'sex', 'VertLevel', and the metric column.
        label_name (str): Name of the metric column to plot (e.g., 'WM').
    """

    df = df.copy()

    # Vertebral levels to plot
    vert_levels = ['3', '4', '5', '6', '7', '3:7']

    # Define colors
    color = "#222020"
    ci_color = 'rgba(34, 32, 32, 0.35)'

    # Initialize figure
    fig = make_subplots(
        rows=len(vert_levels),
        cols=1,
        shared_xaxes=False,
        shared_yaxes=False,
        vertical_spacing=0.05,
        horizontal_spacing=0.1
    )

    for row_idx, vert in enumerate(vert_levels, start=1):

        # Filter data by vertebral level
        data = df[df['VertLevel'] == vert].copy()
        if data.empty:
            continue

        # Fit OLS regression
        x_range = np.linspace(data['age'].min(), data['age'].max(), 100)
        pred_df = pd.DataFrame({'age': x_range})

        metric_col = f'MEAN(area)_{label_name}'
        ols_formula = f'Q("{metric_col}") ~ age'
        ols_model = smf.ols(formula=ols_formula, data=data)
        ols_results = ols_model.fit()

        pred = ols_results.get_prediction(pred_df)
        pred_summary = pred.summary_frame(alpha=0.05)

        y_fit = pred_summary['mean']
        ci_lower = pred_summary['mean_ci_lower']
        ci_upper = pred_summary['mean_ci_upper']

        # Scatter plot of data
        fig.add_trace(
            go.Scatter(
                x=data['age'],
                y=data[metric_col],
                mode='markers',
                marker=dict(color=color, size=11, symbol='circle', opacity=0.8),
                showlegend=False
            ),
            row=row_idx, col=1
        )

        # Fitted regression line
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_fit,
                mode='lines',
                line=dict(color=color, width=2, dash='solid'),
                showlegend=False
            ),
            row=row_idx, col=1
        )

        # Confidence interval shading
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([x_range, x_range[::-1]]),
                y=np.concatenate([ci_upper, ci_lower[::-1]]),
                fill='toself',
                fillcolor=ci_color,
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo='skip',
                showlegend=False
            ),
            row=row_idx, col=1
        )

        # Annotate p-value
        p_value = ols_results.pvalues['age']
        significance = '*' if p_value < 0.05 else ''
        font_color = 'red' if p_value < 0.05 else 'black'

        fig.add_annotation(
            x=0.7,
            y=0.9,
            xref="x domain",
            yref="y domain",
            text=f'p-Age = {p_value:.3f} <span style="color:{font_color}">{significance}</span>',
            showarrow=False,
            font=dict(family='Arial', size=16, color='black'),
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='black',
            borderwidth=1,
            borderpad=4,
            row=row_idx, col=1
        )

        # Update axes
        fig.update_xaxes(
            title_text='Age',
            title_font=dict(family='Arial', size=20, color='black'),
            tickfont=dict(family='Arial', size=18),
            tickvals=list(range(6, 18)),
            range=[5, 18],
            row=row_idx, col=1
        )

        fig.update_yaxes(
            title_text=f'{metric_col} in C{vert}',
            title_font=dict(family='Arial', size=20, color='black'),
            tickfont=dict(family='Arial', size=18),
            row=row_idx, col=1
        )

    # Final layout
    fig.update_layout(
        height=1800,
        width=700,
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff"
    )

    fig.show()

In [58]:
plot_CSA_per_label(df_GM_WM_SC, 'GM')

In [59]:
plot_CSA_per_label(df_GM_WM_SC, 'WM')

In [60]:
plot_CSA_per_label(df_GM_WM_SC, 'SC')

## Plot ratios between GM, WM and SC CSA using spinal levels

In [44]:
def plot_ratio(df, label_1, label_2):

    """
    This function plots the ratio between two metrics (e.g., GM and WM) across different vertebral levels.

    Args:
        df (pd.DataFrame): DataFrame containing the data to plot. Must include columns 'age', 'sex', 'VertLevel', and the ratio column. 
        The ratio column should be named as '{label_1}_{label_2}_ratio'.
        label_1 (str): The first label in the ratio (e.g., 'GM').
        label_2 (str): The second label in the ratio (e.g., 'WM').
    """
    
    df = df.copy()

    # Vertebral levels to plot
    vert_levels = ['3', '4', '5', '6', '7', '3:7']

    # Define color of markers 
    color = "#222020"

    # Color for confidence interval
    ci_color = 'rgba(34, 32, 32, 0.35)' 

    fig = make_subplots(
        rows=6,
        cols=1,
        shared_xaxes=False,
        shared_yaxes=False,
        vertical_spacing=0.05,
        horizontal_spacing=0.1
    )

    for row_idx, vert in enumerate(vert_levels, start=1):

        # Filder data according to the VertLevel
        data = df[df['VertLevel'] == vert].copy()

        print(data)

        x_range = np.linspace(data['age'].min(), data['age'].max(), 100)
        pred_df = pd.DataFrame({'age': x_range})

        ratio_name = f'{label_1}_{label_2}_ratio'
        ols_formula = f'{ratio_name} ~ age'

        ols_model = smf.ols(formula=ols_formula, data=data)
        ols_results = ols_model.fit()

        # Print OLS results
        print(ols_results.summary())

        pred = ols_results.get_prediction(pred_df)
        pred_summary = pred.summary_frame(alpha=0.05)

        y_fit = pred_summary['mean']
        ci_lower = pred_summary['mean_ci_lower']
        ci_upper = pred_summary['mean_ci_upper']

        # Scatter plot
        fig.add_trace(
            go.Scatter(
                x=data['age'],
                y=data[ratio_name],
                mode='markers',
                marker=dict(color=color, size=11, symbol='circle', opacity=0.8),
                showlegend=False
            ),
            row=row_idx, col=1
        )

        # Fit line
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_fit,
                mode='lines',
                line=dict(color=color, width=2, dash='solid'),
                showlegend=False
            ),
            row=row_idx, col=1
        )

        # Confidence interval
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([x_range, x_range[::-1]]),
                y=np.concatenate([ci_upper, ci_lower[::-1]]),
                fill='toself',
                fillcolor=ci_color,
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo='skip',
                showlegend=False,
            ),
            row=row_idx, col=1
        )

        # Add p-value as annotation
        p_value = ols_results.pvalues['age']
        significance = ''
        if p_value < 0.05:
            significance = '*'
            font_color='red'
        else:
            font_color='black'
        fig.add_annotation(
            x=0.7,  # relative within subplot (0–1)
            y=0.9,  # near the top
            xref="x domain",
            yref="y domain",
            text = f'p-Age = {p_value:.3f} <span style="color:{font_color}">{significance}</span>',
            showarrow=False,
            font=dict(family='Arial', size=16, color='black'),
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='black',
            borderwidth=1,
            borderpad=4,
            row=row_idx, col=1
        )

        fig.update_xaxes(
                title_text='Age',
                title_font=dict(family='Arial', size=20, color='black', weight='bold'),
                tickfont=dict(family='Arial', size=18),
                tickvals=list(range(6, 18)),
                range=[5, 18],
                row=row_idx, col=1
            )
        
        fig.update_yaxes(
                title_text=f'{label_1} / {label_2} ratio in C{vert}',
                title_font=dict(family='Arial', size=20, color='black', weight='bold'),
                tickfont=dict(family='Arial', size=18),
                range=[0.6, 2],
                row=row_idx, col=1
            )

    # Update layout
    fig.update_layout(
        height=1800,
        width=700,
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff"
    )


    fig.show()

## Plot GM/WM ratio

In [45]:
plot_ratio(df_GM_WM_SC, 'GM', 'WM')

            Timestamp_WM  SCT Version_WM  \
4    2025-10-20 09:33:33             7.1   
10   2025-10-20 09:30:43             7.1   
16   2025-10-20 09:30:01             7.1   
22   2025-10-20 09:32:45             7.1   
28   2025-10-20 09:34:14             7.1   
34   2025-10-20 09:32:05             7.1   
40   2025-10-20 09:36:29             7.1   
46   2025-10-20 09:32:12             7.1   
52   2025-10-20 09:34:08             7.1   
58   2025-10-20 09:34:52             7.1   
64   2025-10-20 09:36:06             7.1   
70   2025-10-20 09:35:58             7.1   
76   2025-10-20 09:33:26             7.1   
82   2025-10-20 09:31:23             7.1   
88   2025-10-20 09:33:27             7.1   
93   2025-10-20 09:32:05             7.1   
99   2025-10-20 09:38:11             7.1   
105  2025-10-20 09:33:34             7.1   
111  2025-10-20 09:38:50             7.1   
117  2025-10-20 09:36:10             7.1   
123  2025-10-20 09:30:01             7.1   
129  2025-10-20 09:36:45        

## Plot GM/SC ratio

In [46]:
plot_ratio(df_GM_WM_SC, 'GM', 'SC')

            Timestamp_WM  SCT Version_WM  \
4    2025-10-20 09:33:33             7.1   
10   2025-10-20 09:30:43             7.1   
16   2025-10-20 09:30:01             7.1   
22   2025-10-20 09:32:45             7.1   
28   2025-10-20 09:34:14             7.1   
34   2025-10-20 09:32:05             7.1   
40   2025-10-20 09:36:29             7.1   
46   2025-10-20 09:32:12             7.1   
52   2025-10-20 09:34:08             7.1   
58   2025-10-20 09:34:52             7.1   
64   2025-10-20 09:36:06             7.1   
70   2025-10-20 09:35:58             7.1   
76   2025-10-20 09:33:26             7.1   
82   2025-10-20 09:31:23             7.1   
88   2025-10-20 09:33:27             7.1   
93   2025-10-20 09:32:05             7.1   
99   2025-10-20 09:38:11             7.1   
105  2025-10-20 09:33:34             7.1   
111  2025-10-20 09:38:50             7.1   
117  2025-10-20 09:36:10             7.1   
123  2025-10-20 09:30:01             7.1   
129  2025-10-20 09:36:45        

## Plot WM/SC ratio

In [47]:
plot_ratio(df_GM_WM_SC, 'WM', 'SC')

            Timestamp_WM  SCT Version_WM  \
4    2025-10-20 09:33:33             7.1   
10   2025-10-20 09:30:43             7.1   
16   2025-10-20 09:30:01             7.1   
22   2025-10-20 09:32:45             7.1   
28   2025-10-20 09:34:14             7.1   
34   2025-10-20 09:32:05             7.1   
40   2025-10-20 09:36:29             7.1   
46   2025-10-20 09:32:12             7.1   
52   2025-10-20 09:34:08             7.1   
58   2025-10-20 09:34:52             7.1   
64   2025-10-20 09:36:06             7.1   
70   2025-10-20 09:35:58             7.1   
76   2025-10-20 09:33:26             7.1   
82   2025-10-20 09:31:23             7.1   
88   2025-10-20 09:33:27             7.1   
93   2025-10-20 09:32:05             7.1   
99   2025-10-20 09:38:11             7.1   
105  2025-10-20 09:33:34             7.1   
111  2025-10-20 09:38:50             7.1   
117  2025-10-20 09:36:10             7.1   
123  2025-10-20 09:30:01             7.1   
129  2025-10-20 09:36:45        